In [ ]:
import pandas as pd
import paddle
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import math
import os,shutil

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


## 由于操作不当原因，在保存版本之后未能及时保存最优checkpoint文件，十分抱歉！并随项目附上非最优的checkpoint

In [ ]:
def filepreprocess(filename,usage):
    file_handle = open(filename,'r')
    lines = file_handle.readlines()
    rna_list = []
    rna_twodimen = []
    rna_out = []
    match_idx = []
    tmp_rnaout = []
    i=0
    while i<len(lines):
        # print(i)
        if lines[i].find('>id') != -1:
            # print(lines[i])
            tmp_rnaout = []
            tmp_two = []
            i+=1
            tmprna = lines[i].replace('A','0')
            tmprna = tmprna.replace('G','1')
            tmprna = tmprna.replace('C','2')
            tmprna = tmprna.replace('U','3')
            tmprna=tmprna.strip('\n')
            tmprna = [ int(x) for x in list(tmprna)]
            rna_list.append(tmprna)
            # tmprnaprt = linear_rna.linear_partition_v(lines[i].strip('\n'),bp_cutoff=0.7)
            # print(tmprnaprt)
            # return
            i+=1
            tmp_two = lines[i].replace('.','0')
            tmp_two = tmp_two.replace('(','1')
            tmp_two = tmp_two.replace(')','2')
            tmp_two=tmp_two.strip('\n') 
            tmp_two = [ int(x) for x in list(tmp_two)]
            rna_twodimen.append(tmp_two)
            i+=1
            continue
        if usage!='test':
            if lines[i]!='\n':
                # print(lines[i])
            # print(lines[i].split())
                tmp_rnaout.append(float(lines[i].split()[1]))
            else:
                rna_out.append(tmp_rnaout)
                # if usage=='train':
                #     for epoch in range(4):
                #         rna_out.append(tmp_rnaout)
                tmp_rnaout=[]
                tmp_two = []
            i+=1
        else:
            i+=1
    return rna_list,rna_twodimen,rna_out

In [ ]:
def filepreprocessBrank(filename,usage):
    file_handle = open(filename,'r')
    lines = file_handle.readlines()
    rna_list = []
    rna_twodimen = []
    rna_out = []
    match_idx = []
    tmp_rnaout = []
    i=0
    while i<len(lines):
        # print(i)
        if lines[i].find('>') != -1:
            # print(lines[i])
            tmp_rnaout = []
            tmp_two = []
            i+=1
            tmprna = lines[i].replace('A','0')
            tmprna = tmprna.replace('G','1')
            tmprna = tmprna.replace('C','2')
            tmprna = tmprna.replace('U','3')
            tmprna=tmprna.strip('\n')
            tmprna = [ int(x) for x in list(tmprna)]
            rna_list.append(tmprna)
            # tmprnaprt = linear_rna.linear_partition_v(lines[i].strip('\n'),bp_cutoff=0.7)
            # print(tmprnaprt)
            # return
            i+=1
            tmp_two = lines[i].replace('.','0')
            tmp_two = tmp_two.replace('(','1')
            tmp_two = tmp_two.replace(')','2')
            tmp_two=tmp_two.strip('\n') 
            tmp_two = [ int(x) for x in list(tmp_two)]
            # tmp_match_idx = getmatchidx(tmprna,tmp_two)
            # match_idx.append(tmp_match_idx)
            rna_twodimen.append(tmp_two)
            # if usage=='train':
            #     for epoch in range(4):
            #         tmp_seq_auged,tmp_twodimen_auged = dataaugment(tmprna,tmp_two,0.2)
            #         rna_list.append(tmp_seq_auged)
            #         rna_twodimen.append(tmp_twodimen_auged)
            i+=1
            continue
        if usage!='test':
            if lines[i]!='\n':
                # print(lines[i])
            # print(lines[i].split())
                tmp_rnaout.append(float(lines[i].split()[1]))
            else:
                rna_out.append(tmp_rnaout)
                # if usage=='train':
                #     for epoch in range(4):
                #         rna_out.append(tmp_rnaout)
                tmp_rnaout=[]
                tmp_two = []
            i+=1
        else:
            i+=1
    return rna_list,rna_twodimen,rna_out

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

In [ ]:
train_rna_list,train_rna_twodimen,train_rna_out= filepreprocess('data/data68535/train.txt','train')
valid_rna_list,valid_rna_twodimen,valid_rna_out= filepreprocess('data/data68535/dev.txt','valid')

In [ ]:
print(len(train_rna_list))

In [ ]:
from paddle.io import Dataset
class MyDataset(Dataset):
    def __init__(self,maxlen,num_samples,rna_list,rna_twodimen,rna_out):
        super(MyDataset, self).__init__()
        self.maxlen = maxlen
        self.rna_list = rna_list
        self.rna_twodimen = rna_twodimen
        self.rna_out = rna_out
        self.num_samples=num_samples

    def __getitem__(self, index):
        tmplen = len(self.rna_list[index])
        tmprna_sequence = self.rna_list[index]+[4]*(self.maxlen-tmplen)
        tmprna_twodimen = self.rna_twodimen[index]+[3]*(self.maxlen-tmplen)
        # tmprna_sequence = self.rna_list[index]
        # tmprna_twodimen = self.rna_twodimen[index]
        # tmp_match_idx = self.match_idx[index]
        # tmprna_sequence = self.rna_list[index]
        # tmprna_twodimen = self.rna_twodimen[index]
        label = self.rna_out[index]+[0]*(self.maxlen-tmplen)
        # label = self.rna_out[index]
        # rna_sequence_after = []
        # rna_sequence_before = []
        # rna_twodimen_after = []
        # rna_twodimen_before = []
        # rna_jianji = []
        # rna_twodemen = []
        # for j in range(tmplen):
        #     tmp_after_seq = self.rna_list[j+1:]+[4]*(self.maxlen-tmplen-j)
        #     tmp_before_seq = self.rna_list[:j]+[4]*(self.maxlen-j)
        #     tmp_after_twodimen = self.rna_twodimen[j+1:]+[3]*(self.maxlen-tmplen-j)
        #     tmp_before_twodimen = self.rna_twodimen[:j]+[3]*(self.maxlen-j)
        #     tmp_jianji = self.rna_list[j]
        #     tmp_twodimen = self.rna_twodimen[j]
        #     rna_sequence_after.append(tmp_after_seq)
        #     rna_sequence_before.append(tmp_before_seq)
        #     rna_twodimen_after.append(tmp_after_twodimen)
        #     rna_twodimen_before.append(tmp_before_twodimen)
        #     rna_jianji.append(tmp_jianji)
        #     rna_twodemen.append(tmp_twodimen)
        return np.array(tmprna_sequence),np.array(tmprna_twodimen), np.array(label),tmplen
        # return np.array(rna_sequence_after),np.array(rna_sequence_before),np.array(rna_twodimen_after),np.array(rna_sequence_after),np.array(rna_jianji),np.array(rna_twodemen),np.array(label),tmplen

    def __len__(self):
        return self.num_samples


In [ ]:
import paddle.fluid as fluid
import paddle.nn as nn
import paddle
class MyModel(paddle.nn.Layer):
    def __init__(self):
    # 初始化父类中的一些参数
        super(MyModel, self).__init__()
        self.x1embed = paddle.nn.Embedding(num_embeddings=5, embedding_dim=64,padding_idx=4)
        self.x2embed = paddle.nn.Embedding(num_embeddings=4, embedding_dim=64,padding_idx=3)
        self.lstm1 = nn.LSTM(64,32,4,direction='bidirect')
        self.lstm2 = nn.LSTM(128,64,4,direction='bidirect')
        self.lstm4 = nn.LSTM(25,64,4,direction='bidirect')
        # self.lstm2 = nn.LSTM(64,32,4)
        # self.lstm2 = nn.LSTM(32,16,2)
        # self.fc1=paddle.fluid.layers.fc((64,32))
        # self.linear1=nn.Linear(192,64)
        self.linear1=nn.Linear(128,64)
        # self.linear2=nn.Linear(64,32)
        self.linear3 = nn.Linear(64,2)
        self.softmax = nn.Softmax(axis=-1)
        self.relu = nn.LeakyReLU(negative_slope=0.1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x1,x2,x3):
        x1_embed = self.x1embed(x1)
        x2_embed = self.x2embed(x2)
        # x1_embed = paddle.nn.functional.normalize(x1_embed,axis=-1)
        # x1_embed = paddle.fluid.layers.add_position_encoding(x1_embed,alpha=1.0,beta=1.0)
        total1,(_,_)= self.lstm1(x1_embed,sequence_length=x3)
        total = paddle.concat([total1,x2_embed],axis=-1)
        total2,(_,_)=  self.lstm2(total,sequence_length=x3)
        # total2 = paddle.squeeze(total2,axis=0)
        # match_vec = paddle.fluid.layers.gather(total2,x4)
        # total2 = paddle.concat([total2,match_vec],axis=-1)
        # total2 = paddle.unsqueeze(total2,axis=0)
        # total = paddle.concat([total1,total2],axis=-1)
        # total4,(_,_)= self.lstm4(total,sequence_length=x3)
        out = self.linear1(total2)
        out = self.relu(out)
        # out = self.linear2(out)
        # out = self.relu(out)
        out = self.linear3(out)
        # out = self.relu(out)
        out = self.softmax(out)
        return out[:,:,0]


In [ ]:

dataset = MyDataset(500,4750,train_rna_list,train_rna_twodimen,train_rna_out)
validdataset = MyDataset(500,250,valid_rna_list,valid_rna_twodimen,valid_rna_out)

In [ ]:
def train(dataset,epoch_num,valid_rna_list,valid_rna_twodimen,valid_rna_out):
    print('训练开始')
    #实例化模型
    model = MyModel()
    #将模型转换为训练模式
    model.train()
    #设置优化器，学习率，并且把模型参数给优化器
    opt = paddle.optimizer.Adam(learning_rate=0.0001,parameters=model.parameters())
    #设置损失函数
    mse_loss = paddle.nn.MSELoss()
    #设置数据读取器
    data_reader = paddle.io.DataLoader(dataset,
                        batch_size=64,
                        shuffle=True)
    # validdata_reader = paddle.io.DataLoader(validdataset,
    #                     batch_size=10,
    #                     shuffle=True)
    iter_epoch = []
    best_valid_loss = 1000
    best_path = None
    history_loss = []
    # tr_pos_enc = paddle.to_tensor(position_enc(500,128),dtype='float32')
    for epoch in range(epoch_num):
        print(epoch)
        model.train()
        tmp_epoch_loss = []
        for batch_id, data in enumerate(tqdm(data_reader())):
            # print(data)
            x1 = data[0]
            x2 = data[1]
            y = paddle.to_tensor(data[2],dtype='float32')
            x3 = data[3]
            out = model(x1,x2,x3)
            # print(out.shape)
            # print(y.shape)
            # return
            avg_loss= 0
            # print(x3)
            for l in range(x1.shape[0]):
                # print(x3[l].numpy()[0])
                avg_loss += mse_loss(out[l,:int(x3[l].numpy()[0])],y[l,:int(x3[l].numpy()[0])])
            # avg_loss = mse_loss(out,y)
            avg_loss /= x1.shape[0]
            avg_loss.backward()
            tmp_epoch_loss.append(avg_loss.numpy()[0])
            opt.step()
            opt.clear_grad()
        print(f'epoch {epoch}:loss:{np.mean(tmp_epoch_loss)}')
        iter_epoch.append(epoch)
        history_loss.append(np.mean(tmp_epoch_loss))
        model.eval()
        valid_avg_loss = 0
        for i in tqdm(range(len(valid_rna_list))):
            x1 = paddle.to_tensor(np.array(valid_rna_list[i]).reshape((1,-1)),dtype='int32')
            x2 = paddle.to_tensor(np.array(valid_rna_twodimen[i]).reshape((1,-1)),dtype='int32')
            y = paddle.to_tensor(np.array(valid_rna_out[i]),dtype='float32')
            out = model(x1,x2,paddle.to_tensor([len(valid_rna_list[i])]))
            valid_avg_loss+= mse_loss(out,y)
            # valid_avg_loss/=x1.shape[0]
        tmp_valid_loss = valid_avg_loss.numpy()[0]/len(valid_rna_list)
        print(f'epoch {epoch}:val_loss:{tmp_valid_loss}')
        if tmp_valid_loss<best_valid_loss:
            best_valid_loss = tmp_valid_loss
            if best_path is not None:
                if os.path.exists(best_path):
                        os.remove(best_path)
            best_path=f'pretrain/model_val_{best_valid_loss}'
            paddle.save(model.state_dict(),best_path)
    #绘制loss
    plt.plot(iter_epoch,history_loss, label = 'loss')
    plt.legend()
    plt.xlabel('iters')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
train(dataset,12,valid_rna_list,valid_rna_twodimen,valid_rna_out)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dataloader/dataloader_iter.py:89: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if isinstance(slot[0], (np.ndarray, np.bool, numbers.Number)):
  4%|▍         | 3/75 [00:01<00:32,  2.19it/s]

0


  0%|          | 0/250 [00:00<?, ?it/s]/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:143: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
  1%|          | 2/250 [00:00<00:22, 11.01it/s]

epoch 0:loss:0.19863837957382202


  6%|▌         | 14/250 [00:01<00:17, 13.75it/s] 51%|█████     | 128/250 [00:08<00:08, 14.56it/s]


In [ ]:
test_rna_list,test_rna_twodimen,test_rna_out= filepreprocessBrank('data/data68535/B_board_112_seqs.txt','test')

In [ ]:
def test(test_rna_list,test_rna_twodimen):
    print('预测开始')
    #实例化模型
    model = MyModel()
    layer_state_dict = paddle.load('pretrain/model3.717')
    model.load_dict(layer_state_dict)
    model.eval()   # 预测
    total_loss = []
    datas = []
    # 预测所有正常时序
    mse_loss = paddle.nn.loss.MSELoss()
    # 这里设置batch_size为1，单独求得每个数据的loss
    for i in tqdm(range(len(test_rna_list))):
        # pos_enc = paddle.to_tensor(position_enc(len(test_rna_list[i]),128),dtype='float32')
        # print(len(test_rna_list[i]))
        # print(len(test_rna_twodimen[i]))
        out = model(paddle.Tensor(np.array([test_rna_list[i]]).reshape((1,-1))),paddle.Tensor(np.array([test_rna_twodimen[i]]).reshape((1,-1))),paddle.to_tensor([len(test_rna_list[i])]))
        out = out.numpy()
        out = np.reshape(out,(-1,1))
        np.savetxt(f'predictB/{i+1}.predict.txt',out,fmt='%.04f')

In [ ]:
test(test_rna_list,test_rna_twodimen)

  1%|          | 1/112 [00:00<00:16,  6.67it/s]

预测开始


  2%|▏         | 2/112 [00:00<00:16,  6.69it/s]100%|██████████| 112/112 [00:10<00:00, 10.31it/s]


In [ ]:
data = paddle.to_tensor([1, 2, 3], dtype='int32')
out = paddle.expand(data, shape=[1,2, 3])
print(out)

Tensor(shape=[1, 2, 3], dtype=int32, place=CUDAPlace(0), stop_gradient=True,
       [[[1, 2, 3],
         [1, 2, 3]]])
